In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256, 121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44, 121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66, 230), thickness=2, circle_radius=2),
                             )
    

In [10]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)  as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        

        draw_landmarks(image, results)
        
    
        cv2.imshow("OpenCv", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### Extract Keypoints

In [37]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, lh, rh])

### Setup Folders for Collection

In [40]:
DATA_PATH = os.path.join("MP_DATA")

# Actions to detect
actions = np.array(['hello', 'thanks', 'i love you'])

# Thirty videos worth of data
no_sequences = 30

# No of frames in a video 
sequence_length = 30

In [42]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

### Collect Keypoint Values for Training and Testing

In [43]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)  as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                

                ret, frame = cap.read()

                # make detection
                image, results = mediapipe_detection(frame, holistic)


                draw_landmarks(image, results)
                
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    cv2.imshow("OpenCv", image)
                    cv2.waitKey(2000)
                    
                else:
                    cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow("OpenCv", image)
                    
                    
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                
                    

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()


### Preprocess Data and Create labels and Features

In [45]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [46]:
label_map = {label:num for num, label in enumerate(actions)}

In [47]:
label_map

{'hello': 0, 'thanks': 1, 'i love you': 2}

In [49]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        

In [51]:
X = np.array(sequences)

In [52]:
X.shape

(90, 30, 1662)

In [54]:
y = to_categorical(labels).astype(int)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

### Build and Train LSTM Neural Network

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [59]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [62]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [63]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [64]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 4s 96ms/step - loss: 4.8405 - categorical_accuracy: 0.2471
Epoch 2/2000
3/3 [==============================] - 0s 94ms/step - loss: 23.3299 - categorical_accuracy: 0.5176
Epoch 3/2000
3/3 [==============================] - 0s 90ms/step - loss: 54.9824 - categorical_accuracy: 0.4824
Epoch 4/2000
3/3 [==============================] - 0s 84ms/step - loss: 66.2238 - categorical_accuracy: 0.3882
Epoch 5/2000
3/3 [==============================] - 0s 87ms/step - loss: 102.6677 - categorical_accuracy: 0.3294
Epoch 6/2000
3/3 [==============================] - 0s 88ms/step - loss: 66.3616 - categorical_accuracy: 0.3765
Epoch 7/2000
3/3 [==============================] - 0s 89ms/step - loss: 32.8716 - categorical_accuracy: 0.3529
Epoch 8/2000
3/3 [==============================] - 0s 84ms/step - loss: 26.1802 - categorical_accuracy: 0.2941
Epoch 9/2000
3/3 [==============================] - 0s 107ms/step - loss: 11.2590 - categorical_accuracy

Epoch 74/2000
3/3 [==============================] - 0s 89ms/step - loss: 0.6288 - categorical_accuracy: 0.6353
Epoch 75/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.5846 - categorical_accuracy: 0.6471
Epoch 76/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.5867 - categorical_accuracy: 0.6588
Epoch 77/2000
3/3 [==============================] - 0s 83ms/step - loss: 1.2290 - categorical_accuracy: 0.5294
Epoch 78/2000
3/3 [==============================] - 0s 89ms/step - loss: 1.1201 - categorical_accuracy: 0.4706
Epoch 79/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.7156 - categorical_accuracy: 0.7059
Epoch 80/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.7340 - categorical_accuracy: 0.8118
Epoch 81/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.7470 - categorical_accuracy: 0.8118
Epoch 82/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.7219 - categorical_accuracy:

3/3 [==============================] - 0s 93ms/step - loss: 0.1515 - categorical_accuracy: 0.9529
Epoch 147/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.1414 - categorical_accuracy: 0.9529
Epoch 148/2000
3/3 [==============================] - 0s 82ms/step - loss: 0.1402 - categorical_accuracy: 0.9529
Epoch 149/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.1048 - categorical_accuracy: 0.9647
Epoch 150/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.1130 - categorical_accuracy: 0.9647
Epoch 151/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.0945 - categorical_accuracy: 0.9765
Epoch 152/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.1106 - categorical_accuracy: 0.9529
Epoch 153/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0803 - categorical_accuracy: 0.9765
Epoch 154/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.0915 - categorical_accuracy: 0.976

3/3 [==============================] - 0s 85ms/step - loss: 0.1128 - categorical_accuracy: 0.9647
Epoch 219/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.1084 - categorical_accuracy: 0.9412
Epoch 220/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0766 - categorical_accuracy: 0.9765
Epoch 221/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.1082 - categorical_accuracy: 0.9647
Epoch 222/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.1000 - categorical_accuracy: 0.9529
Epoch 223/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0979 - categorical_accuracy: 0.9529
Epoch 224/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.0741 - categorical_accuracy: 0.9765
Epoch 225/2000
3/3 [==============================] - 0s 89ms/step - loss: 0.1034 - categorical_accuracy: 0.9765
Epoch 226/2000
3/3 [==============================] - 0s 82ms/step - loss: 0.0942 - categorical_accuracy: 0.964

3/3 [==============================] - 0s 85ms/step - loss: 0.0391 - categorical_accuracy: 0.9882
Epoch 291/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.0389 - categorical_accuracy: 0.9882
Epoch 292/2000
3/3 [==============================] - 0s 92ms/step - loss: 0.0391 - categorical_accuracy: 0.9882
Epoch 293/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.0388 - categorical_accuracy: 0.9882
Epoch 294/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.0385 - categorical_accuracy: 0.9882
Epoch 295/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.0383 - categorical_accuracy: 0.9882
Epoch 296/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.0380 - categorical_accuracy: 0.9882
Epoch 297/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.0379 - categorical_accuracy: 0.9882
Epoch 298/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.0378 - categorical_accuracy: 0.988

...

3/3 [==============================] - 0s 82ms/step - loss: 7.1481e-06 - categorical_accuracy: 1.0000
Epoch 1697/2000
3/3 [==============================] - 0s 86ms/step - loss: 7.1312e-06 - categorical_accuracy: 1.0000
Epoch 1698/2000
3/3 [==============================] - 0s 85ms/step - loss: 7.1144e-06 - categorical_accuracy: 1.0000
Epoch 1699/2000
3/3 [==============================] - 0s 84ms/step - loss: 7.0934e-06 - categorical_accuracy: 1.0000
Epoch 1700/2000
3/3 [==============================] - 0s 83ms/step - loss: 7.0751e-06 - categorical_accuracy: 1.0000
Epoch 1701/2000
3/3 [==============================] - 0s 86ms/step - loss: 7.0611e-06 - categorical_accuracy: 1.0000
Epoch 1702/2000
3/3 [==============================] - 0s 100ms/step - loss: 7.0485e-06 - categorical_accuracy: 1.0000
Epoch 1703/2000
3/3 [==============================] - 0s 87ms/step - loss: 7.0289e-06 - categorical_accuracy: 1.0000
Epoch 1704/2000
3/3 [==============================] - 0s 122ms/step - 

3/3 [==============================] - 0s 81ms/step - loss: 6.0696e-06 - categorical_accuracy: 1.0000
Epoch 1766/2000
3/3 [==============================] - 0s 81ms/step - loss: 6.0626e-06 - categorical_accuracy: 1.0000
Epoch 1767/2000
3/3 [==============================] - 0s 81ms/step - loss: 6.0500e-06 - categorical_accuracy: 1.0000
Epoch 1768/2000
3/3 [==============================] - 0s 84ms/step - loss: 6.0304e-06 - categorical_accuracy: 1.0000
Epoch 1769/2000
3/3 [==============================] - 0s 81ms/step - loss: 6.0234e-06 - categorical_accuracy: 1.0000
Epoch 1770/2000
3/3 [==============================] - 0s 94ms/step - loss: 6.0107e-06 - categorical_accuracy: 1.0000
Epoch 1771/2000
3/3 [==============================] - 0s 86ms/step - loss: 5.9953e-06 - categorical_accuracy: 1.0000
Epoch 1772/2000
3/3 [==============================] - 0s 90ms/step - loss: 5.9785e-06 - categorical_accuracy: 1.0000
Epoch 1773/2000
3/3 [==============================] - 0s 85ms/step - lo

3/3 [==============================] - 0s 87ms/step - loss: 5.2100e-06 - categorical_accuracy: 1.0000
Epoch 1835/2000
3/3 [==============================] - 0s 82ms/step - loss: 5.1946e-06 - categorical_accuracy: 1.0000
Epoch 1836/2000
3/3 [==============================] - 0s 83ms/step - loss: 5.1847e-06 - categorical_accuracy: 1.0000
Epoch 1837/2000
3/3 [==============================] - 0s 82ms/step - loss: 5.1721e-06 - categorical_accuracy: 1.0000
Epoch 1838/2000
3/3 [==============================] - 0s 84ms/step - loss: 5.1665e-06 - categorical_accuracy: 1.0000
Epoch 1839/2000
3/3 [==============================] - 0s 83ms/step - loss: 5.1525e-06 - categorical_accuracy: 1.0000
Epoch 1840/2000
3/3 [==============================] - 0s 85ms/step - loss: 5.1455e-06 - categorical_accuracy: 1.0000
Epoch 1841/2000
3/3 [==============================] - 0s 82ms/step - loss: 5.1343e-06 - categorical_accuracy: 1.0000
Epoch 1842/2000
3/3 [==============================] - 0s 86ms/step - lo

3/3 [==============================] - 0s 85ms/step - loss: 4.4765e-06 - categorical_accuracy: 1.0000
Epoch 1904/2000
3/3 [==============================] - 0s 83ms/step - loss: 4.4723e-06 - categorical_accuracy: 1.0000
Epoch 1905/2000
3/3 [==============================] - 0s 82ms/step - loss: 4.4555e-06 - categorical_accuracy: 1.0000
Epoch 1906/2000
3/3 [==============================] - 0s 84ms/step - loss: 4.4457e-06 - categorical_accuracy: 1.0000
Epoch 1907/2000
3/3 [==============================] - 0s 83ms/step - loss: 4.4373e-06 - categorical_accuracy: 1.0000
Epoch 1908/2000
3/3 [==============================] - 0s 84ms/step - loss: 4.4317e-06 - categorical_accuracy: 1.0000
Epoch 1909/2000
3/3 [==============================] - 0s 82ms/step - loss: 4.4176e-06 - categorical_accuracy: 1.0000
Epoch 1910/2000
3/3 [==============================] - 0s 83ms/step - loss: 4.4064e-06 - categorical_accuracy: 1.0000
Epoch 1911/2000
3/3 [==============================] - 0s 83ms/step - lo

3/3 [==============================] - 0s 83ms/step - loss: 3.8651e-06 - categorical_accuracy: 1.0000
Epoch 1973/2000
3/3 [==============================] - 0s 84ms/step - loss: 3.8525e-06 - categorical_accuracy: 1.0000
Epoch 1974/2000
3/3 [==============================] - 0s 84ms/step - loss: 3.8469e-06 - categorical_accuracy: 1.0000
Epoch 1975/2000
3/3 [==============================] - 0s 85ms/step - loss: 3.8399e-06 - categorical_accuracy: 1.0000
Epoch 1976/2000
3/3 [==============================] - 0s 81ms/step - loss: 3.8286e-06 - categorical_accuracy: 1.0000
Epoch 1977/2000
3/3 [==============================] - 0s 85ms/step - loss: 3.8188e-06 - categorical_accuracy: 1.0000
Epoch 1978/2000
3/3 [==============================] - 0s 82ms/step - loss: 3.8160e-06 - categorical_accuracy: 1.0000
Epoch 1979/2000
3/3 [==============================] - 0s 85ms/step - loss: 3.8090e-06 - categorical_accuracy: 1.0000
Epoch 1980/2000
3/3 [==============================] - 0s 83ms/step - lo

In [65]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_3 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [91]:
res = model.predict(X_test)[0]

In [92]:
res

array([6.4898854e-19, 2.3380089e-09, 1.0000000e+00], dtype=float32)

In [89]:
actions[np.argmax(res)]

'hello'

In [74]:
y_test

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [75]:
model.save('action.h5')

In [76]:
# model.load_weights('action.h5')

In [77]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [78]:
yhat = model.predict(X_test)

In [79]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [80]:
ytrue

[2, 0, 0, 0, 0]

In [81]:
yhat

[2, 0, 0, 0, 0]

In [83]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 0],
        [0, 4]],

       [[4, 0],
        [0, 1]]])

In [84]:
accuracy_score(ytrue, yhat)

1.0

### Test in real time

In [94]:
cap = cv2.VideoCapture(0)

sequence = []
sentence = []
threshold = 0.4

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)  as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        

        draw_landmarks(image, results)
        
        
        # prediction login
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0 ]
            print(actions[np.argmax(res)])
        
        
        # viz logic
    
        cv2.imshow("OpenCv", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

i love you
thanks
thanks
i love you
i love you
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love you
i love yo